In [ ]:
import xarray as xr
import numpy as np
from math import pi
import datetime as dt
import os
import sys
from os.path import exists
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from datetime import *; from dateutil.relativedelta import *
from scipy.interpolate import make_interp_spline, BSpline
from scipy.signal import savgol_filter
import pandas as pd
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False})
dir_timeseries = 'F:/data/project_data/NASA_biophysical/UnimakPass/data/'

#unimak pass center point
ulon = -165.125
ulat = 54.236
ilen=14245
time = pd.date_range("1982-01-01", periods=ilen, freq='D')
da = xr.DataArray(np.ones((ilen))*np.nan,dims=["time"],coords={'time':time})
ds_bird = xr.Dataset({'tem':da})

time = pd.date_range("1982-01-01", periods=ilen, freq='D')
lat,lon = np.arange(ulat-1,ulat+1,.25),np.arange(ulon-1,ulon+1,.25)
da = xr.DataArray(np.ones((8,8,ilen))*np.nan,dims=['lat','lon','time'],coords={'lat':lat,'lon':lon,'time':time})
ds_bird_area=xr.Dataset({'temp':da})

time = pd.date_range("1982-01-01", periods=ilen, freq='D')
lat,lon = np.arange(ulat-2,ulat+2,.25),np.arange(ulon-2,ulon+2,.25)
da = xr.DataArray(np.ones((16,16,ilen))*np.nan,dims=['lat','lon','time'],coords={'lat':lat,'lon':lon,'time':time})
ds_bird_area2=xr.Dataset({'temp':da})

time = pd.date_range("1982-01-01", periods=ilen, freq='D')
lat,lon = np.arange(ulat-4,ulat+4,.25),np.arange(ulon-4,ulon+4,.25)
da = xr.DataArray(np.ones((32,32,ilen))*np.nan,dims=['lat','lon','time'],coords={'lat':lat,'lon':lon,'time':time})
ds_bird_area4=xr.Dataset({'temp':da})

In [ ]:
#define function to get all the data at once, use same years for climatology for all data
def get_data():
    
    #climatology years
    cyr1,cyr2='1993-01-01','2018-12-31'
    
    # CCMP test
    dir_pattern_zarr = 'F:/data/sat_data/ccmp/zarr/'
    ds= xr.open_zarr(dir_pattern_zarr)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds_ccmp = ds.sortby(ds.lon)
    ds_ccmp = ds_ccmp.drop('nobs')
    for var in ds_ccmp:
        tem = ds_ccmp[var].attrs
        tem['var_name']='ccmp_'+str(var)
        ds_ccmp[var].attrs=tem
    #ds_ccmp_clim = ds_ccmp.sel(time=slice(cyr1,cyr2))
    #ds_ccmp_clim = ds_ccmp_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    # AVISO test
    dir_pattern_zarr = 'F:/data/sat_data/aviso/aviso.zarr/'
    ds= xr.open_zarr(dir_pattern_zarr)
    ds = ds.rename({'latitude':'lat','longitude':'lon'})
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds_aviso = ds.sortby(ds.lon).drop({'lat_bnds','lon_bnds','crs','err'})
    for var in ds_aviso:
        tem = ds_aviso[var].attrs
        tem['var_name']='aviso_'+str(var)
        ds_aviso[var].attrs=tem
    #ds_aviso_clim = ds_aviso.sel(time=slice(cyr1,cyr2))
    #ds_aviso_clim = ds_aviso_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)    

    #sst
    dir_pattern_zarr = 'F:/data/sat_data/sst/cmc/zarr/'
    ds_sst= xr.open_zarr(dir_pattern_zarr)
    ds_sst = ds_sst.drop({'analysis_error','mask','sea_ice_fraction'})
    tem = ds_sst.analysed_sst.attrs
    tem['var_name']='cmc_sst'
    ds_sst.analysed_sst.attrs=tem
    #ds_sst_clim = ds_sst.sel(time=slice(cyr1,cyr2))
    #ds_sst_clim = ds_sst_clim.groupby('time.dayofyear').mean('time',keep_attrs=True,skipna=False)
    
    #color
    ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-8DAY') #CCI_ALL-v4.2-DAILY')
    #ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-DAILY')
    #ds_color = xr.open_dataset('https://rsg.pml.ac.uk/thredds/dodsC/CCI_ALL-v4.2-8DAY')
    for var in ds_color:
        if not var=='chlor_a':
            ds_color = ds_color.drop(var)
    ds_color = ds_color.sortby('lat')
    for var in ds_color:
        tem = ds_color[var].attrs
        tem['var_name']='CCI_'+str(var)
        ds_color[var].attrs=tem

    
    #get bathymetry from ETOPO1
    fname_topo = 'F:/data/topo/ETOPO1_Ice_g_gmt4.grd'
    ds = xr.open_dataset(fname_topo)
    ds_topo = ds.rename_dims({'x':'lon','y':'lat'}).rename({'x':'lon','y':'lat'})
    tem = ds_topo.z.attrs
    tem['var_name']='etopo_depth'
    ds_topo.z.attrs=tem

    #put data into a dictionary
    data_dict={'aviso':ds_aviso,
               'wnd':ds_ccmp,
               'sst':ds_sst,
              'topo':ds_topo,
              'color':ds_color}
  
    return data_dict

In [ ]:
data = get_data()

In [ ]:
ds = ds_bird
ds['ccmp_wnd']=np.sqrt(ds.ccmp_uwnd**2+ds.ccmp_vwnd**2)
clim_dy = ds.groupby("time.dayofyear").mean("time")
anom_dy = ds.groupby("time.dayofyear") - clim_dy
clim_mn = ds.groupby("time.month").mean("time")
anom_mn = ds.groupby("time.month") - clim_mn

In [ ]:
ds=anom_dy.drop('tem')
f, ax = plt.subplots(11, 1, figsize=(6, 20))
for ivar,var in enumerate(ds):
    print(ivar,var)
    ds[var].plot(ax=ax[ivar])

In [ ]:
for name in data:
    ds_data=data[name]
    if name=='topo':
        continue
    print('name',name)
    lat1,lat2=ulat-2,ulat+2
    lon1,lon2=ulon-2,ulon+2
    tem = ds_data.sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    tem = tem.interp(lat=ds_bird_area.lat,lon=ds_bird_area.lon,time=ds_bird_area.time,method='nearest')
    for var in ds_data:
        var_tem=ds_data[var].attrs['var_name']
        ds_bird_area[var_tem]=tem[var]
        ds_bird_area[var_tem].attrs=ds_data[var].attrs
fname_out = 'unimak_ts_area'
ds_bird_area = ds_bird_area.drop('temp').reset_coords()
#add a land mask, only cci has it, so use the cci to create a land mask & apply to all variables
ds_bird_area = ds_bird_area.where(ds_bird_area.CCI_chlor_a.sum('time')>0)
ds_bird_area.to_netcdf(dir_timeseries+fname_out+'_1deg.nc')

for name in data:
    ds_data=data[name]
    if name=='topo':
        continue
    print('name',name)
    lat1,lat2=ulat-3,ulat+3
    lon1,lon2=ulon-3,ulon+3
    tem = ds_data.sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    tem = tem.interp(lat=ds_bird_area2.lat,lon=ds_bird_area2.lon,time=ds_bird_area2.time,method='nearest')
    for var in ds_data:
        var_tem=ds_data[var].attrs['var_name']
        ds_bird_area2[var_tem]=tem[var]
        ds_bird_area2[var_tem].attrs=ds_data[var].attrs
fname_out = 'unimak_ts_area'
ds_bird_area2 = ds_bird_area2.drop('temp').reset_coords()
ds_bird_area2 = ds_bird_area2.where(ds_bird_area2.CCI_chlor_a.sum('time')>0)
ds_bird_area2.to_netcdf(dir_timeseries+fname_out+'_2deg.nc')

for name in data:
    ds_data=data[name]
    if name=='topo':
        continue
    print('name',name)
    lat1,lat2=ulat-5,ulat+5
    lon1,lon2=ulon-5,ulon+5
    tem = ds_data.sel(lat=slice(lat1,lat2),lon=slice(lon1,lon2)).load()
    tem = tem.interp(lat=ds_bird_area4.lat,lon=ds_bird_area4.lon,time=ds_bird_area4.time,method='nearest')
    for var in ds_data:
        var_tem=ds_data[var].attrs['var_name']
        ds_bird_area4[var_tem]=tem[var]
        ds_bird_area4[var_tem].attrs=ds_data[var].attrs
fname_out = 'unimak_ts_area'
ds_bird_area4 = ds_bird_area4.drop('temp').reset_coords()
ds_bird_area4 = ds_bird_area4.where(ds_bird_area4.CCI_chlor_a.sum('time')>0)
ds_bird_area4.to_netcdf(dir_timeseries+fname_out+'_4deg.nc')

# test data

In [ ]:
fname_out = 'unimak_ts_area_4deg'
fname = dir_timeseries+fname_out+'.nc'
print(fname)
ds = xr.open_dataset(fname)
ds.close()
ds['wnd']=np.sqrt(ds.ccmp_uwnd**2+ds.ccmp_vwnd**2)
ds

In [ ]:
#ds.CCI_chlor_a.sum('time').plot()
ds.ccmp_vwnd.sel(time='2017-01-14').plot()
ds.cmc_sst.sel(time='2017-01-14').plot()
#ds.CCI_chlor_a[10000,:,:].plot()
#plt.scatter(ds.lon[16],ds.lat[14])
#ds.ccmp_vwnd[:,15,14].plot()

In [ ]:
tem = ds.sel(time='2017-01-14')
tem.wnd[:,:].plot()
plt.scatter(tem.lon[15],tem.lat[14])

In [ ]:
clim_dy = ds.groupby("time.dayofyear").mean("time")
anom_dy = ds.groupby("time.dayofyear") - clim_dy
clim_mn = ds.groupby("time.month").mean("time")
anom_mn = ds.groupby("time.month") - clim_mn

In [ ]:
#anom_dy.ccmp_vwnd.sel(time='2017-01-14').plot()
anom_dy.cmc_sst.sel(time='2017-01-14').plot()

In [ ]:
anom_dy.ccmp_vwnd[:,15,14].plot()
anom_dy.cmc_sst[:,15,14].plot()

In [ ]:
#ds.ccmp_vwnd[:,15,14].plot()
clim_mn.ccmp_vwnd[:,15,14].plot()
#anom_mn.ccmp_vwnd[:,15,14].sel(time=slice('2014','2017')).plot()

In [ ]:
import datashader
plt.scatter(anom_dy.ccmp_vwnd,anom_dy.cmc_sst)